<h1> Delay management <h1>

    The objective of this dashboard is to perform some analysis about the rentals returned late like:
    - How often are drivers late for the next check-in? How does it impact the next driver?
    - How is the delay repartition between the different check-in type?
    And explore the possibility of a new feature, a threshold between the location to avoid late rentals, answering questions like:
    - Which share of our owner’s revenue would potentially be affected by the feature? How many rentals would be affected by the feature depending on the threshold and scope we choose?
    - How long should the minimum delay be?
    - should we enable the feature for all cars?, only Connect cars?
    - How many problematic cases will it solve depending on the chosen threshold and scope?

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

<h2> Upload and take a first look at the dataset <h2>

In [2]:
df_delay = pd.read_excel('get_around_delay_analysis.xlsx', index_col=None)

In [3]:
print(f"Number of rows : {df_delay.shape[0]}")
print(f"Number of columns : {df_delay.shape[1]}")
print()

print("Display of df_delay: ")
display(df_delay.head(10))
print()

print("Basics statistics: ")
display(df_delay.describe(include='all'))
print()
pd.options.display.max_rows = 999
print("Percentage of missing values: ")
display((100 * df_delay.isnull().sum() /df_delay.shape[0]).sort_values(ascending= False))

Number of rows : 21310
Number of columns : 7

Display of df_delay: 


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN
5,511626,398802,mobile,ended,-203.0,NaN,NaN
6,511639,370585,connect,ended,-15.0,563782.0,570.0
7,512303,371242,mobile,ended,-44.0,NaN,NaN
8,512475,322502,mobile,canceled,NaN,NaN,NaN
9,513434,256528,connect,ended,23.0,NaN,NaN



Basics statistics: 


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000



Percentage of missing values: 


previous_ended_rental_id                      91.360863
time_delta_with_previous_rental_in_minutes    91.360863
delay_at_checkout_in_minutes                  23.294228
rental_id                                      0.000000
car_id                                         0.000000
checkin_type                                   0.000000
state                                          0.000000
dtype: float64

There is a lot of missing values in 'previous_ended_rental_id' and 'time_delta_with_previous_rental_in_minutes', which is logical as not every location followed another.

Sort by delay at checkout

In [4]:
df_delay.sort_values(by = 'delay_at_checkout_in_minutes', ascending= False)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
1021,532240,379042,mobile,ended,71084.0,NaN,NaN
1787,549980,402829,mobile,ended,64059.0,NaN,NaN
13776,537546,408750,mobile,ended,37382.0,NaN,NaN
1038,534699,270195,mobile,ended,23126.0,NaN,NaN
5257,556557,413591,mobile,ended,22618.0,NaN,NaN
...,...,...,...,...,...,...,...
21291,570189,257207,mobile,ended,NaN,NaN,NaN
21297,571481,311841,mobile,canceled,NaN,NaN,NaN
21300,572293,355023,connect,canceled,NaN,NaN,NaN
21302,573285,394863,mobile,ended,NaN,NaN,NaN


<h2> EDA <h2>

<h2> Late rental analysis <h2>

Proportion of late checkout

In [5]:
mask = df_delay['delay_at_checkout_in_minutes'] > 0
df_delay['checkout_status'] = ["Late" if x>0 else "in_time" for x in df_delay['delay_at_checkout_in_minutes']] 
fig = px.pie(df_delay, names='checkout_status')
fig.update_layout(title_text='Proportion of late checkout',title_y=0.5)
fig.show()

Mean checkout delay

In [6]:
print('Average checkout time:')
(df_delay.loc[:,'delay_at_checkout_in_minutes']).mean()

Average checkout time:


59.70151719075003

Mean checkout delay when late

In [7]:
print('Average delay time when late:')
(df_delay.loc[mask, 'delay_at_checkout_in_minutes']).mean()

Temps de retard moyen lors de retard:


201.79147171416417

Max delay checkout

In [8]:
print('Maximal delay:')
(df_delay.loc[:, 'delay_at_checkout_in_minutes']).max()

Temps de retard maximum:


71084.0

Mean delay time between two locations

In [9]:
print('Average time between two rentals:')
(df_delay.loc[:,'time_delta_with_previous_rental_in_minutes']).mean()

Délai moyen entre deux locations:


279.28843020097776

<h4> Late proportion by checkin type <h4>

In [10]:
df_delay['delay_repartition'] = 'In time'
df_delay.loc[((df_delay['checkin_type'] == 'mobile') & (df_delay['checkout_status'] == 'Late')),'delay_repartition'] = 'Mobile and late'
df_delay.loc[((df_delay['checkin_type'] == 'connect') & (df_delay['checkout_status'] == 'Late')),'delay_repartition'] = 'Connect and late'
fig = px.pie(df_delay, names='delay_repartition', title='Proportion of late checkout', width= 700)
fig.update_layout(title_text='Proportion of late checkout',title_y=0.95,title_x=0.5)
fig.show()

33.87508706756443
46.72704816797036


It seems that there is way more late rental in proportion when the check-in type is mobile, but it might be that there is juste more mobile check-in, how is it when we look at the percentage:

In [ ]:
Late_perc_mobile = len(df_delay.loc[((df_delay['checkin_type'] == 'mobile') & (df_delay['checkout_status'] == 'Late')),:])/len(df_delay.loc[(df_delay['checkin_type'] == 'mobile'),:])*100
Late_perc_connect = len(df_delay.loc[((df_delay['checkin_type'] == 'connect') & (df_delay['checkout_status'] == 'Late')),:])/len(df_delay.loc[(df_delay['checkin_type'] == 'connect'),:])*100
print(Late_perc_connect)
print(Late_perc_mobile)

As we can see the mobile type of check in also got an higher proportion of late checkout of 13%, to finaly help decide if there is a need to add a threshold only for one type of check-in type (like the connect) we can check the average delay time by check-in type:

Mean late when the checkin is done by mobile

In [11]:
mask2 = (df_delay['delay_at_checkout_in_minutes'] > 0) & (df_delay['checkin_type'] == 'mobile')
print('Average delay time by mobile check-in (when late) :')
(df_delay.loc[mask2, 'delay_at_checkout_in_minutes']).mean()

Temps de retard moyen lors de retard:


224.13681560730018

Mean late when the checkin is done by connect

In [12]:
mask2 = (df_delay['delay_at_checkout_in_minutes'] > 0) & (df_delay['checkin_type'] == 'connect')
print('Average delay time by mobile check-in (when late) :')
(df_delay.loc[mask2, 'delay_at_checkout_in_minutes']).mean()

Temps de retard moyen lors de retard:


80.10966415352982

Considering this data it could be intresting for the product Management team to consider creating a threshold concerning at least the people that check-in by mobile.

<h4> Late checkout impact evaluation <h4>

Construction of a dataset matching the rented car with the previous rental

In [13]:
df_loc_consecutive = pd.merge(df_delay, df_delay, how='inner', left_on = 'previous_ended_rental_id', right_on = 'rental_id')


df_loc_consecutive.drop(
    [
        "delay_at_checkout_in_minutes_x",
        "rental_id_y", 
        "car_id_y", 
        "state_y",
        "time_delta_with_previous_rental_in_minutes_y",
        "previous_ended_rental_id_y",
        "checkout_status_x"
    ], 
    axis=1,
    inplace=True
)

df_loc_consecutive = df_loc_consecutive.rename(columns={
    'rental_id_x' : 'rental_id',
    'car_id_x': 'car_id',
    'checkin_type_x':'checkin_type',
    'state_x':'state',
    'previous_ended_rental_id_x':'previous_ended_rental_id',
    'time_delta_with_previous_rental_in_minutes_x':'time_delta_with_previous_rental_in_minutes',
    'checkin_type_y':'previous_checkin_type',
    'delay_at_checkout_in_minutes_y':'previous_delay_at_checkout_in_minutes',
    'checkout_status_y':'previous_checkout_status'
})
df_loc_consecutive.head()

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_repartition_x,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status,delay_repartition_y
0,511639,370585,connect,ended,563782.0,570.0,In time,connect,136.0,Late,Connect and late
1,519491,312389,mobile,ended,545639.0,420.0,Mobile and late,mobile,140.0,Late,Mobile and late
2,521156,392479,mobile,ended,537298.0,0.0,In time,mobile,NaN,in_time,In time
3,525044,349751,mobile,ended,510607.0,60.0,In time,mobile,-113.0,in_time,In time
4,528808,181625,connect,ended,557404.0,330.0,In time,connect,-352.0,in_time,In time


drop of the rows with missing values

In [14]:
mask = df_loc_consecutive["previous_delay_at_checkout_in_minutes"].notnull() 
df_loc_consecutive = df_loc_consecutive.loc[mask, :]
df_loc_consecutive.reset_index(drop=True, inplace=True)
print('Number of total rentals:{}'.format(len(df_delay)))
print('Number of consecutive rentals:{}'.format(len(df_loc_consecutive)))

Nombre de location totale:21310
Nombre de location consécutive:1729


Calculation of the real delay between the locations taking into account rentals returned late

In [15]:
df_loc_consecutive['real_delay_between_loc_in_min'] = df_loc_consecutive['time_delta_with_previous_rental_in_minutes'] - df_loc_consecutive['previous_delay_at_checkout_in_minutes']
df_loc_consecutive.sort_values(by = 'real_delay_between_loc_in_min')

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_repartition_x,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status,delay_repartition_y,real_delay_between_loc_in_min
278,548388,297511,mobile,ended,546459.0,420.0,Mobile and late,mobile,12968.0,Late,Mobile and late,-12548.0
1535,564699,415572,mobile,ended,565228.0,240.0,Mobile and late,mobile,4258.0,Late,Mobile and late,-4018.0
512,541178,392635,mobile,ended,523175.0,0.0,Mobile and late,mobile,2976.0,Late,Mobile and late,-2976.0
1082,573931,300377,mobile,ended,568712.0,120.0,In time,mobile,1760.0,Late,Mobile and late,-1640.0
650,545099,389599,mobile,ended,539042.0,0.0,Mobile and late,mobile,1500.0,Late,Mobile and late,-1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1570,557930,400886,mobile,ended,553320.0,450.0,Mobile and late,mobile,-1414.0,in_time,In time,1864.0
553,567806,216701,mobile,canceled,567644.0,690.0,In time,mobile,-1408.0,in_time,In time,2098.0
1628,564111,369648,connect,ended,561113.0,420.0,Connect and late,connect,-2109.0,in_time,In time,2529.0
1198,570888,265239,connect,ended,551401.0,270.0,In time,connect,-2938.0,in_time,In time,3208.0


Determination of the impacted location

In [16]:
#Impacted location, when the late returned is superior to the time planned between the two rentals 
df_impacted_loc = df_loc_consecutive.loc[df_loc_consecutive['real_delay_between_loc_in_min'] < 0, :]
#Impacted location that have been canceled
df_impacted_canceled_loc = df_impacted_loc.loc[df_impacted_loc['state'] == 'canceled',:]
#Location non impacted by a late return
df_non_impacted_loc = df_loc_consecutive.loc[df_loc_consecutive['real_delay_between_loc_in_min'] >= 0, :]
#Total number of canceled location
total_cancel = len(df_loc_consecutive.loc[(df_loc_consecutive['state'] == 'canceled')])
#Number of rental cancel that have not been impacted by late return
no_late_cancel = len(df_non_impacted_loc.loc[(df_non_impacted_loc['state'] == 'canceled')])

print('Number of rentals impacted: {}'.format(len(df_impacted_loc)))
print('Number of canceled rentals with a delay: {}'.format(len(df_impacted_canceled_loc)))
print("Percentage of cancellations without apparent reasons: {}".format(no_late_cancel/len(df_non_impacted_loc) *100))
print("Percentage of rentals canceled when late: {}".format(len(df_impacted_canceled_loc)/len(df_impacted_loc)*100))
print("We notice a higher percentage of canceled rentals when the delay of a rental impacts the next rental")


Nombre de location impactées: 218
Nombre de location annulées présentant un retard: 37
Pourcentage d'annulation sans raisons apparentes: 11.184645929847784
Pourcentage de location annulées lors de retard: 16.972477064220186
On remarque un pourcentage de location annulées supérieurs lorsque le retard d'une location impact la location suivante
On peut estimer le nombre de locaton annulées dû à ce retard à: 33 (nombre de location impactées par retard moins le pourcentage de location annulées sans raison)


Percentage of impacted location:

In [17]:
percent_impacted_loc = len(df_impacted_loc)/len(df_delay) *100
percent_impacted_loc

1.022993899577663

Percentage of rentals canceled (with every location comprised impacted and non impacted) due to this delay

In [18]:
#Number of rentals impacted and canceled less the 11% which are representative of rentals canceled without reason divide by the total number of rentals
percent_cancel_late = (len(df_impacted_canceled_loc)- (len(df_impacted_canceled_loc)*11/100))/len(df_delay)*100
percent_cancel_late

0.15452839042702954

Late checkout proportion with impacting delay added

In [20]:
df_impacted_loc['previous_ended_rental_id'] = df_impacted_loc['previous_ended_rental_id'].apply(lambda x: int(x))

for elt in df_delay['rental_id']:
    if elt in (df_impacted_loc['previous_ended_rental_id'].values):
        df_delay.loc[elt,'checkout_status'] = 'late and impacting'
        
fig = px.pie(df_delay, names='checkout_status', title='Proportion of late checkout', width = 700)
fig.update_layout(title_text='Proportion of late checkout',title_y=0.95,title_x=0.5)
fig.show()

In [21]:
df_loc_consecutive.sort_values(by = 'real_delay_between_loc_in_min')

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,delay_repartition_x,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status,delay_repartition_y,real_delay_between_loc_in_min
278,548388,297511,mobile,ended,546459.0,420.0,Mobile and late,mobile,12968.0,Late,Mobile and late,-12548.0
1535,564699,415572,mobile,ended,565228.0,240.0,Mobile and late,mobile,4258.0,Late,Mobile and late,-4018.0
512,541178,392635,mobile,ended,523175.0,0.0,Mobile and late,mobile,2976.0,Late,Mobile and late,-2976.0
1082,573931,300377,mobile,ended,568712.0,120.0,In time,mobile,1760.0,Late,Mobile and late,-1640.0
650,545099,389599,mobile,ended,539042.0,0.0,Mobile and late,mobile,1500.0,Late,Mobile and late,-1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1570,557930,400886,mobile,ended,553320.0,450.0,Mobile and late,mobile,-1414.0,in_time,In time,1864.0
553,567806,216701,mobile,canceled,567644.0,690.0,In time,mobile,-1408.0,in_time,In time,2098.0
1628,564111,369648,connect,ended,561113.0,420.0,Connect and late,connect,-2109.0,in_time,In time,2529.0
1198,570888,265239,connect,ended,551401.0,270.0,In time,connect,-2938.0,in_time,In time,3208.0


The number of location impacted by late rental is quite small and can question the necessity of a threshold, we will study further the threshold management in the next part:

<h1> Threshold <h1>

The problematic is difficult to approach, as a threshold will indeed prevent the late location to impact other, it also make in every cars (which include those who would not be unavailble due to delay) unavaible for an extra time, and therefore losing potential client and profit.

Therefore we need to find a good balance in order to avoid a maximum of the friction but also having the smaller threshold possible to avoid making the cars unavaible for too long.

<h4> Delay repartition overview <h4> 

In [23]:
df_impacted_canceled_loc = df_impacted_canceled_loc.reset_index()
df_impacted_canceled_loc = df_impacted_canceled_loc.drop('index', axis = 1)

In [24]:
fig2= px.bar(df_impacted_canceled_loc, x= 'real_delay_between_loc_in_min', y = df_impacted_canceled_loc.index, title = 'Delay repartition overview' )
fig2.update_layout(
    xaxis_title = 'Delay time exceeding on the next rental',
    yaxis_title = 'Number of rental',
    title={
        'text': 'Delay repartition overview',
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},)
fig2.show()

<h4> Threshold simulation all cars: <h4>

In [25]:
def threshold_result(max_threshold):
    late_avoided = []
    location_missed = []
    for threshold in range(0,max_threshold):
        df_loc_consecutive_threshold = df_loc_consecutive.copy()
        df_loc_consecutive_threshold['time_delta_with_previous_rental_in_minutes'] = df_loc_consecutive_threshold['time_delta_with_previous_rental_in_minutes'].apply(lambda x: threshold if x < threshold else x )
        df_loc_consecutive_threshold['real_delay_between_loc_in_min'] = df_loc_consecutive_threshold['time_delta_with_previous_rental_in_minutes'] - df_loc_consecutive_threshold['previous_delay_at_checkout_in_minutes']
        df_impacted_loc_theshold = df_loc_consecutive_threshold.loc[df_loc_consecutive_threshold['real_delay_between_loc_in_min'] < 0, :]
        late_avoided.append(len(df_impacted_loc) - len(df_impacted_loc_theshold))
        location_missed.append(len(df_loc_consecutive) - len(df_loc_consecutive.loc[df_loc_consecutive['time_delta_with_previous_rental_in_minutes'] > threshold,:]))
    fig_threshold = px.line(x = [i for i in range(0,max_threshold)], y = [late_avoided,location_missed] )
    legend_names = {'wide_variable_0':'Late location avoided', 'wide_variable_1': 'Location missed'}
    fig_threshold.for_each_trace(lambda t: t.update(name = legend_names[t.name],
                                      legendgroup = legend_names[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, legend_names[t.name])
                                     ))
    fig_threshold.update_layout(
    height=700, 
    xaxis_title = 'Threshold value',
    title={
        'text': 'Threshold impact',
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
)
    fig_threshold.show()

threshold_result(500)

<h4> Threshold simulation mobile connect: <h4>

In [26]:
def threshold_result(max_threshold):
    late_avoided = []
    location_missed = []
    for threshold in range(0,max_threshold):
        df_loc_consecutive_threshold = df_loc_consecutive.loc[df_loc_consecutive['previous_checkin_type'] == 'mobile', : ]
        df_loc_consecutive_threshold['time_delta_with_previous_rental_in_minutes'] = df_loc_consecutive_threshold['time_delta_with_previous_rental_in_minutes'].apply(lambda x: threshold if x < threshold else x )
        df_loc_consecutive_threshold['real_delay_between_loc_in_min'] = df_loc_consecutive_threshold['time_delta_with_previous_rental_in_minutes'] - df_loc_consecutive_threshold['previous_delay_at_checkout_in_minutes']
        df_impacted_loc_theshold = df_loc_consecutive_threshold.loc[df_loc_consecutive_threshold['real_delay_between_loc_in_min'] < 0, :]
        late_avoided.append(len(df_impacted_loc) - len(df_impacted_loc_theshold))
        location_missed.append(len(df_loc_consecutive) - len(df_loc_consecutive.loc[df_loc_consecutive['time_delta_with_previous_rental_in_minutes'] > threshold,:]))
    fig_threshold = px.line(x = [i for i in range(0,max_threshold)], y = [late_avoided,location_missed] )
    legend_names = {'wide_variable_0':'Late location avoided', 'wide_variable_1': 'Location missed'}
    fig_threshold.for_each_trace(lambda t: t.update(name = legend_names[t.name],
                                      legendgroup = legend_names[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, legend_names[t.name])
                                     ))
    fig_threshold.update_layout(
    height=700, 
    xaxis_title = 'Threshold value',
    title={
        'text': 'Threshold impact',
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
)
    fig_threshold.show()

threshold_result(500)

C:\Users\flori\AppData\Local\Temp\ipykernel_22168\2583441211.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\flori\AppData\Local\Temp\ipykernel_22168\2583441211.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



As we can see on the graph (when we set a high threshold number like 500 to get an overview) and as we could see on the preview most of the delay can be avoid with a threshold around 200 minutes,
after this the increase in the late location avoided become very slow, while the possible missed location due to the increase of time (and therefore indisponibility between two location) keep increasing.

Considering that even with a threshold around 200 the missed location is significatively higher (906) than the late location avoided (180) the necessity of a threshold can be discussed.
The management team must determined if the frictions (and their consequences) caused by the delay are more important than the high number of missed location induce by a threshold.